## Import statements

In [ ]:
from {{cookiecutter.project_slug}}.util.config import cfg
from {{cookiecutter.project_slug}}.util.data_accessors import snowflake_dataframe_from_sql, snowflake_execute_query

%matplotlib inline 
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from pandas_profiling import ProfileReport
!jupyter nbextension enable --py widgetsnbextension

## Data Ingestion

In [ ]:
df = snowflake_dataframe_from_sql("Select * FROM stage_js.datascience.store_ratings")

## Data Processing

In [ ]:
common_timestamp_columns = ['date', 'created_at','updated_at','dt','timestamp','event_time']
for column in df.columns:
    if str(column).lower() in common_timestamp_columns:
        print(f"Updating {column} from object to datetime")
        df[column] = pd.to_datetime(df[column])

## EDA

Basic column information and counts

In [ ]:
df.info()

##### Data profiling
This can be written to disk using the to_file(filename) command or it can be displayed directly

In [ ]:
pr = ProfileReport(df, explorative = True)
pr.to_file("00-EDA-template-profile.html")

#### Pairwise analysis

In [ ]:
# Set the default theme for seaborn
sns.set_theme()
sns.pairplot(df[df.columns], hue="total_score")